In [10]:
# Update and import packages
!pip install -Uqr requirements.txt
!/bin/bash ./local_mode_setup.sh

# Basic packages
import json
import pandas as pd
pd.set_option("display.max_colwidth", None)

# AWS packages
import sagemaker
from sagemaker.sklearn import SKLearn
from sagemaker.predictor import Predictor
import boto3

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
./local_mode_setup.sh: line 3: sudo: command not found
The user does not have root access. Everything required to run the notebook is already installed and setup. We are good to go!


In [19]:
# Train SM estimator
train_data = 's3://amplifyobserverinsights-aoinsightslandingbucket29-5vcr471d4nm5/data/issues/'
sklearn_estimator = SKLearn(
    'train_tfidf.py',
    role=sagemaker.get_execution_role(),
#     instance_type='local',
    instance_type='ml.m5.4xlarge',
    framework_version='0.23-1',
    hyperparameters = {'n-best': 10},
    dependencies=['.']
)
sklearn_estimator.fit({'train': train_data})

2021-08-02 21:51:33 Starting - Starting the training job...
2021-08-02 21:51:56 Starting - Launching requested ML instancesProfilerReport-1627941090: InProgress
...
2021-08-02 21:52:29 Starting - Preparing the instances for training.........
2021-08-02 21:54:03 Downloading - Downloading input data
2021-08-02 21:54:03 Training - Downloading the training image...
2021-08-02 21:54:20 Training - Training image download completed. Training in progress.2021-08-02 21:54:20,813 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2021-08-02 21:54:20,816 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-08-02 21:54:20,824 sagemaker_sklearn_container.training INFO     Invoking user training script.
2021-08-02 21:54:21,723 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/miniconda3/bin/python -m pip install -r requirements.txt
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=

In [22]:
# Deploy SM predictor
# predictor = sklearn_estimator.deploy(instance_type='ml.m5.4xlarge',
#                                      initial_instance_count=1,
# #                                      update_endpoint=True,
#                                     endpoint_name='issue-similarity-endpoint')
predictor = Predictor('issue-similarity-endpoint')
model = sklearn_estimator.create_model()
session = model.sagemaker_session
role = sagemaker.get_execution_role()
model_name = model.name
container_def = model.prepare_container_def(instance_type='ml.m5.4xlarge')
session.create_model(model_name, role, container_def)


endpoint_config_name = session.create_endpoint_config(name=model_name,
                                                      model_name=model_name,
                                                      initial_instance_count=1,
                                                      instance_type='ml.m5.4xlarge')

# Update desired endpoint with new Endpoint Config
client = boto3.client('sagemaker')
client.update_endpoint(EndpointName='issue-similarity-endpoint',
                       EndpointConfigName=endpoint_config_name)

# predictor.update_endpoint(
#     instance_type='ml.m5.4xlarge',
#     model_name = sklearn_estimator.create_model().name,
#     initial_instance_count=1
# )

{'EndpointArn': 'arn:aws:sagemaker:us-west-2:092109498566:endpoint/issue-similarity-endpoint',
 'ResponseMetadata': {'RequestId': 'c3f8b6d2-9cf1-46b3-81ce-e4176cb89255',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c3f8b6d2-9cf1-46b3-81ce-e4176cb89255',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '93',
   'date': 'Mon, 02 Aug 2021 22:04:33 GMT'},
  'RetryAttempts': 0}}

In [ ]:
data = ['password manager autofill']
data = json.dumps({'data': data})
for i in range(1000):
    response = predictor.predict(data)
print(response)